In [1]:
import config
import event
import handler
import strategy
import portfolio
import risk
import analysis as an
from datetime import datetime


#SQL & API credentials. Store in config file.

access_token = config.access_token
account_id = config.account_id

sql_host = config.sql_host
sql_user = config.sql_user_analysis
sql_password = config.sql_password_analysis

In [2]:
dh = handler.DataHandler()
queue = event.EventQueue()
strategy = strategy.Strategy()
risk = risk.Risk('USDJPY', 2, 2)
portfolio = portfolio.Portfolio()

In [3]:
queue.add_to_queue(dh.get_single_rate())

In [4]:
while len(queue.queue) != 0:
    next_event = queue.next_event()
    
    if next_event.type == 'tick':
        #Call strategy
        queue.add_to_queue(strategy.generate_signal(next_event))
        print('Tick Event Popped')

    elif next_event.type == 'signal':
        # If signal requires, call risk
        if next_event.signal != 'none':
            queue.add_to_queue(risk.calculate_risk(next_event))
            print('Signal Event Popped')

    elif next_event.type == 'trade':
        #Call portfolio
        print('Trade Event Popped')

    elif next_event.type == 'order':
        #Call execution
        print('Order Event Popped')

    elif next_event.type == 'fill':
        #Wait in execution function until response received
        #Call portfolio
        print('Fill Event Popped')

    else:
        print('Queue Error')


Tick Event Popped
Signal Event Popped
Trade Event Popped


In [5]:
len(queue.queue)

0

In [6]:
print(next_event.time)
print(next_event.signal)
print(next_event.sl)
print(next_event.tp)
next_event.price

2017-05-31 16:38:57.527913
sell
111.286
111.086


{'ask': 111.196, 'bid': 111.21600000000001}

In [7]:
risk.position_size_calculator(500, .01, 'buy', next_event.price, 111.286)

500.0

In [8]:
risk.margin_requirement(500, .02)

10.0

In [9]:
next_event.pip_stop_loss

7.0

In [10]:
res = portfolio.account_summary()

In [11]:
resp = res.json()

In [12]:
resp

{'account': {'NAV': '500.0000',
  'alias': 'Nov16mt',
  'balance': '500.0000',
  'commission': '0',
  'createdByUserID': 1551265,
  'createdTime': '2016-11-08T19:09:21.311863625Z',
  'currency': 'GBP',
  'financing': '0',
  'hedgingEnabled': False,
  'id': '101-004-1551265-001',
  'lastTransactionID': '3',
  'marginAvailable': '500.0000',
  'marginCallMarginUsed': '0.0000',
  'marginCallPercent': '0.00000',
  'marginCloseoutMarginUsed': '0.0000',
  'marginCloseoutNAV': '500.0000',
  'marginCloseoutPercent': '0.00000',
  'marginCloseoutPositionValue': '0.0000',
  'marginCloseoutUnrealizedPL': '0.0000',
  'marginRate': '0.02',
  'marginUsed': '0.0000',
  'openPositionCount': 0,
  'openTradeCount': 0,
  'orders': [],
  'pendingOrderCount': 0,
  'pl': '0.0000',
  'positionValue': '0.0000',
  'positions': [],
  'resettablePL': '0.0000',
  'trades': [],
  'unrealizedPL': '0.0000',
  'withdrawalLimit': '500.0000'},
 'lastTransactionID': '3'}

In [14]:
res

<Response [200]>